# Script para tratamentos dos dados de texto

In [1]:
# Manipulação de Dados
import numpy as np
import pandas as pd

# Data Mining
import nltk
import re
import string

# Pré-processamento e Machine Learning
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
df = pd.read_csv('olist_order_reviews_dataset.csv')

In [3]:
# Dropa os valores em nulos da variável "review_comment_message"
df.dropna(subset=['review_comment_message'], inplace=True)

In [4]:
df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16 00:00:00,2018-02-20 10:52:22
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01


In [5]:
# Remove os emoji
df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,nan,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,nan,Parabns lojas lannister adorei comprar pela In...,2018-03-01 00:00:00,2018-03-02 10:26:53
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
12,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,nan,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16 00:00:00,2018-02-20 10:52:22
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confivel, produto ok e entrega antes ...",2018-05-23 00:00:00,2018-05-24 03:00:01
...,...,...,...,...,...,...,...
99983,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,nan,Entregou dentro do prazo. O produto chegou em ...,2018-02-07 00:00:00,2018-02-19 19:47:23
99990,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,nan,"O produto no foi enviado com NF, no existe ven...",2018-05-19 00:00:00,2018-05-20 21:51:06
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,nan,"Excelente mochila, entrega super rpida. Super ...",2018-03-22 00:00:00,2018-03-23 09:10:43
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,nan,Solicitei a compra de uma capa de retrovisor c...,2017-12-15 00:00:00,2017-12-16 01:29:43


In [6]:
# Cria a classe com todos os procedimentos de limpeza necessários para os reviews
class CleanReview(BaseEstimator, TransformerMixin):
    
    # Converte texto em letras minúsculas
    def to_lower(self, reviews_text):
        return reviews_text.lower()
    
    # Remove a pontuação
    def remove_dig(self, reviews_text):
        return re.sub('\d+', '', reviews_text)
    
    # Remove a pontuação
    def remove_punct(self, reviews_text):
        # Cria uma tabela de tradução
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Todo simbolo da pontuação e substituido por um espaço
        return reviews_text.translate(trantab)
    
    # Define o treinamento
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, **transform_params):
        clean = X.apply(
            self.to_lower).apply(
            self.remove_dig).apply(
            self.remove_punct)
        return clean

In [7]:
# Aplica as função
cr = CleanReview()
reviews_clean = cr.fit_transform(df['review_comment_message'])
reviews_clean = pd.DataFrame(reviews_clean)
reviews_clean.head()

,review_comment_message
3,recebi bem antes do prazo estipulado
4,parabéns lojas lannister adorei comprar pela i...
9,aparelho eficiente no site a marca do aparelh...
12,mas um pouco travando pelo valor ta boa \r\n
15,vendedor confiável produto ok e entrega antes...


In [8]:
reviews_clean['order_id'] = df['order_id']

In [9]:
reviews_clean.head()

,review_comment_message,order_id
3,recebi bem antes do prazo estipulado,658677c97b385a9be170737859d3511b
4,parabéns lojas lannister adorei comprar pela i...,8e6bfb81e283fa7e4f11123a3fb894f1
9,aparelho eficiente no site a marca do aparelh...,b9bf720beb4ab3728760088589c62129
12,mas um pouco travando pelo valor ta boa \r\n,9d6f15f95d01e79bd1349cc208361f09
15,vendedor confiável produto ok e entrega antes...,e51478e7e277a83743b6f9991dbfa3fb


In [10]:
reviews_clean.shape

(41753, 2)

In [11]:
reviews_clean.to_csv('reviews_clean.csv')